In [326]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import re

In [2]:
driver_path = 'D:/Users/figohjs/Documents/WebScraping/chromedriver.exe'
website = "https://www.sc.com.my/regulation/enforcement/have-you-seen-these-people"

In [554]:
options = webdriver.ChromeOptions()
options.add_argument('headless')   
# options.add_argument("window-size=1200x600")
# driver = webdriver.Chrome(executable_path = driver_path, options = options)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)
driver.get(website)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [D:\Users\figohjs\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [588]:
def extractInfo(text):
    tempDict = {}
    infoTextList = text.split('\n')

    #extract name
    name = re.search("(.*)\'s", infoTextList[0], re.I).group(1)
    tempDict['Name'] = name

    #extract ic
    nric = infoTextList[2]
    tempDict['NRIC'] = nric

    #track last element
    lastElementList = [re.search("(.*please contact Securities Commission Malaysia at:)", i).group(1) for i in infoTextList 
                       if re.search("please contact Securities Commision Malaysia", i)]
    if len(lastElementList)>0:
        lastElement = lastElementList[0]
        notImportantIndex = infoTextList.index(lastElement)
    else:
        notImportantIndex = -1

    #if business location and residential address are in popout-content
    if re.search('Last Known Business', ''.join(infoTextList)) \
    and re.search('Last Known Residential Address', ''.join(infoTextList)):
        businessLocIndex = infoTextList.index('Last Known Business / Employment address:')
        residentLocIndex = infoTextList.index('Last Known Residential Address:')
        #extract business location
        businessLoc = ' '.join(infoTextList[businessLocIndex + 1:residentLocIndex])
        #extract residential location
        residentLoc = ' '.join(infoTextList[residentLocIndex + 1:notImportantIndex])

    #if business location is not in popout-content
    if not re.search('Last Known Business', ''.join(infoTextList)) \
    and re.search('Last Known Residential Address', ''.join(infoTextList)):
        residentLocIndex = infoTextList.index('Last Known Residential Address:')
        #extract business location
        businessLoc = ''
        #extract residential location
        residentLoc = ' '.join(infoTextList[residentLocIndex + 1:notImportantIndex])    

    #clean up
    businessLoc = re.sub("ADDRESS [12] ", '', businessLoc)
    tempDict['BusinessLoc'] = businessLoc

    #clean up
    residentLoc = re.sub("ADDRESS [12] ", '', residentLoc)
    tempDict['ResidentLoc'] = residentLoc
    
    return tempDict

### Debug

In [54]:
#Persons Wanted by the SC and List of People Sought to Assist in Investigation
className = "atom-235e66ed-8b22-4a94-a258-017e3fc3abad"
oneDiv =  driver.find_elements_by_xpath("//div[contains(@class, '%s')]//div"%className)

#get class name for both section
twoClassNames = [i.get_attribute("class") for i in oneDiv if re.search("row row", i.get_attribute("class"))]

#start from image
# allImg = driver.find_elements_by_xpath("//div[contains(@class, '%s')]//img[contains(@class, 'so-thumbnail')]"%twoClassNames[1])
# allSrc = [i.get_attribute("src") for i in allImg]
# #filter src with most-wanted-person
# allSrcFilter = [i for i in allSrc if not re.search("most-wanted-person", i)]

In [598]:
#wanted list
#all class for a tag
xPath = "//div[@class = '%s']//a"%twoClassNames[0]
allClassAFirstSec = [i.get_attribute("class") for i in driver.find_elements_by_xpath(xPath)]

#all src attributes for img tag
xPath = "//div[@class = '%s']//img[@class = 'so-thumbnail']"%twoClassNames[0]
allImgFirstSec = driver.find_elements_by_xpath(xPath)
allImgSrcFirstSec = [i.get_attribute('src') for i in allImgFirstSec]

#store result
infoList = []
for no, imageSrc in enumerate(allImgSrcFirstSec):
    print(imageSrc)
    driver.get(website)
    
    xPath1 = "//a[@class = '%s']/img[@src = '%s']"%(allClassAFirstSec[no], imageSrc)
    wait = WebDriverWait(driver, 30)
    # auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath1)))
    auto_element = wait.until(EC.element_to_be_clickable((By.XPATH, xPath1)))
    # auto_element.click()
    driver.execute_script("arguments[0].click();", auto_element)
    
    xPath2 = "//div[contains(@class, 'mfp-content')]"
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    element = WebDriverWait(driver, 20, ignored_exceptions = ignoredExceptions)\
                            .until(EC.presence_of_element_located((By.XPATH, xPath2)))
    info = element.find_element_by_xpath(xPath2)
    result = extractInfo(info.text)
    infoList.append(result)

https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/img-most-wanted-person/alice-poh.jpg
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/icons/icon-more-info-grey.png
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/img-most-wanted-person/goh-bak-ming.jpg
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/icons/icon-more-info-grey.png
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/img-most-wanted-person/wanted-person-wong-shee-kai.jpg
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/icons/icon-more-info-grey.png
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/regulation/enforcement/uzir2021.png
https://www.sc.com.my/clients/Secur

In [599]:
dfWanted = pd.DataFrame(infoList).drop_duplicates()
dfWanted['Category'] = "WantedList"

In [600]:
dfWanted

,Name,NRIC,BusinessLoc,ResidentLoc,Category
0,ALICE POH GAIK LYE,550608-07-5550,N/A,"7 Jalan SS7/3, Kelana Jaya 47301 Petaling Jaya...",WantedList
2,GOH BAK MING,431127-10-5569,N/A,3 Jalan 16/14 Section 16 46350 Petaling Jaya S...,WantedList
4,WONG SHEE KAI (RICKY),811212-14-5957,"Asia Media Group Berhad No. 35, First Floor, J...","No. 5, Jalan Enak 2, Taman Gembira, 58200 Kual...",WantedList
6,UZIR BIN ABDUL SAMAD,720824-01-5543,"UAS Bistari Management Sdn Bhd, No. 11A, Jalan...","No. 14, Jalan SS 17/2A, 47500 Subang Jaya, Sel...",WantedList
8,TAN WHAI OON / JIGME PHENDE,600910-07-5001,"SJ Asset Management Sdn Bhd, Suite 12-10 & 12-...","No. 1, Jalan 16/6, Seksyen 16, 46350 Petaling ...",WantedList
10,"WONG JIT KIANG, DANIEL",551227-01-6359 (new) / 4905358 (old),,"9A-B, Impiana Condo, Jalan Ulu Klang, 68000 Am...",WantedList
12,YEOH GUAN YONG,761125-07-5011,Remisier’s Assistant of Hwang DBS Securities S...,"No. 556-A, Jln. Abdullah Ariff, 11400 Ayer Ita...",WantedList
14,"YEO WEE SIONG, GEORGE",S7022197G (Singapore),"Megan Media Holdings Berhad, No.4 Jalan SS13/5...","806 Bedok Resevoir Road, #0402, Singapore 4792...",WantedList
16,JENNIFER TAY SIEW KIEM,6353775,"S.K.Tay & Co. Suite, 19-04-01 Aras 4, Plaza Ke...","116-19-8, Jalan Sultan Abdul Samad, Brickfield...",WantedList


In [601]:
#sought to assist
#all class for a tag
xPath = "//div[@class = '%s']//a"%twoClassNames[1]
allClassAFirstSec = [i.get_attribute("class") for i in driver.find_elements_by_xpath(xPath)]

#all src attributes for img tag
xPath = "//div[@class = '%s']//img[@class = 'so-thumbnail']"%twoClassNames[1]
allImgFirstSec = driver.find_elements_by_xpath(xPath)
allImgSrcFirstSec = [i.get_attribute('src') for i in allImgFirstSec]

infoList = []
for no, imageSrc in enumerate(allImgSrcFirstSec):
    print(imageSrc)
    driver.get(website)
    
    xPath1 = "//a[@class = '%s']/img[@src = '%s']"%(allClassAFirstSec[no], imageSrc)
    wait = WebDriverWait(driver, 30)
    # auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath1)))
    auto_element = wait.until(EC.element_to_be_clickable((By.XPATH, xPath1)))
    # auto_element.click()
    driver.execute_script("arguments[0].click();", auto_element)
    
    xPath2 = "//div[contains(@class, 'mfp-content')]"
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    element = WebDriverWait(driver, 20, ignored_exceptions = ignoredExceptions)\
                            .until(EC.presence_of_element_located((By.XPATH, xPath2)))
    info = element.find_element_by_xpath(xPath2)
    result = extractInfo(info.text)
    infoList.append(result)

https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/img-most-wanted-person/wanted-person-azhidi.jpg
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/icons/icon-more-info-grey.png
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/img-most-wanted-person/fauziah-cropped.jpg
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/icons/icon-more-info-grey.png
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/img-most-wanted-person/ong-tai-chin_cropped.jpg
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/icons/icon-more-info-grey.png
https://www.sc.com.my/clients/Security_Commission_FFFF0951-2947-47E1-93FE-FFDB029E57D9/contentms/img/img-most-wanted-person/avatar.png
https://www.sc.com.my/clients/

In [602]:
dfAssist = pd.DataFrame(infoList).drop_duplicates()
dfAssist['Category'] = "SoughtToAssist"

In [603]:
dfAssist

,Name,NRIC,BusinessLoc,ResidentLoc,Category
0,"HSU, CHIN-SHUI",300420231,"Crest Focus Sdn Bhd (583510-P), Plot 19, Free ...","22, Jalan Pantai Jerjak 18, Sungai Dua, 11900 ...",SoughtToAssist
1,MOHD AZHIDI BIN LAILI,751207-08-5177,,No. 65 Lorong 2 Taman Jengka Mahkota 2 26400 B...,SoughtToAssist
2,LO CHOK PING,691124-08-5305,"35A, Jalan Medan Ipoh 6, Bandar Baru Medan, 31...","10, Laluan Ipoh Permai 11, Taman Ipoh Indah, 3...",SoughtToAssist
3,FAUZIAH BINTI MAHFODZ,681209-01-5796,N/A,"No 72, Jalan PI 4/8, Taman Pulai Indah, Skudai...",SoughtToAssist
4,GEH CHOH HENG (TERENCE),710905-10-5453,"1Malaysia Development Berhad, Level 5, Menara ...","27 Jalan Setiamurni Satu, Damansara Heights, 5...",SoughtToAssist
5,ONG TAI CHIN,520924-02-5793,"Astral Supreme Berhad No. B-02-06, Plaza Daman...","80, Taman Bukit Indah Kuah Langkawi, Kedah 239...",SoughtToAssist
6,LOW CHUN CHAI,540105-10-5747,N/A,"48, Kg Baru Kalumpang 44100 Kerling, Selangor ...",SoughtToAssist
15,HADIJAH BINTI BACHIK,630326-06-5346,"Danau Azam Sdn Bhd 41, Kampung Bentung 26500 M...","No. 461, Lorong Satu, Kampung Berkat, 26500 Ma...",SoughtToAssist
16,RUSLI BIN JAAFAR,760320-03-5125,"CTB Solutions Sdn Bhd No. 35, Third Floor Jala...","No. 311, Block 2, Pangsapuri Rimau Indah, Jala...",SoughtToAssist
18,SAHALUDDIN BIN MAT ISA,760528-08-6319,"CTB Solutions Sdn Bhd No. 35, Third Floor Jala...","Block B-T07-U02, Jalan P/8B, Presint 18R13, 62...",SoughtToAssist


In [604]:
#Combine table
dfResult = pd.concat([dfWanted, dfAssist]).copy()
csvFile = 'D:/Users/figohjs/Documents/WebScraping/sc/2021-02-24_SC_wantedList.csv'
dfResult.to_csv(csvFile, index = False)